<a href="https://colab.research.google.com/github/LbFARES/Overview-paper/blob/main/Text_classification_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


Implement a Transformer block as a layer


In [3]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)  # eplison is a small float added to variance to avoid dividing by zero
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

        # rate It represents the fraction of the input units to drop, in our case 10% of of the neurons in this layer will be randomly dropped in each epoch

        self.dropout1 = layers.Dropout(rate) 
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Implement embedding layer


In [4]:

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim) #Turn positive integers (indexes) into dense vectors of fixed size.
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

Download and prepare dataset


In [8]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
#print(x_train.shape)
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen) #pad the input into 2D array
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)
#print(x_train.shape)


25000 Training sequences
25000 Validation sequences


Create classifier model using transformer layer


In [14]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)
#outputs =layers.Dense((1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [15]:
import time


Train and Evaluate


In [16]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])




In [17]:

# Grab Currrent Time Before Running the Code
start = time.time()
history = model.fit(
    x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val)
)

# Grab Currrent Time After Running the Code
end = time.time()
#Subtract Start Time from The End Time
total_time = end - start
print("\n"+ str(total_time))

Epoch 1/3
782/782 [==============================] - 121s 152ms/step - loss: 0.3995 - accuracy: 0.8033 - val_loss: 0.2901 - val_accuracy: 0.8760
Epoch 2/3
782/782 [==============================] - 109s 139ms/step - loss: 0.1993 - accuracy: 0.9249 - val_loss: 0.3106 - val_accuracy: 0.8710
Epoch 3/3
782/782 [==============================] - 109s 139ms/step - loss: 0.1344 - accuracy: 0.9512 - val_loss: 0.3688 - val_accuracy: 0.8588

383.45198822021484
